In [50]:
from psiresp.tests.datafiles import DMSO, DMSO_O1, DMSO_O2
import numpy as np

In [9]:
import psi4
import qcelemental as qcel

In [5]:
def coordinates_from_xyzfile(file):
    return np.loadtxt(file, skiprows=2, usecols=(1, 2, 3), comments='!')


def psi4mol_from_xyzfile(file):
    with open(file, "r") as f:
        xyz = f.read()
    mol = psi4.core.Molecule.from_string(xyz, dtype="xyz", fix_com=True,
                                         fix_orientation=True)
    mol.update_geometry()
    mol.activate_all_fragments()
    return mol

In [6]:
mol = psi4mol_from_xyzfile(DMSO)

In [19]:
mol.units()

'Angstrom'

In [8]:
np.array(mol.geometry())

array([[  7.1407494 ,   1.34160357, -15.94150286],
       [  7.02969573,   3.27934588, -15.29029187],
       [  6.37365911,   0.06141398, -14.54136782],
       [  6.08497659,   1.14334413, -17.68106387],
       [ 10.36395491,   0.51005289, -16.60560116],
       [ 10.38402733,  -2.23374996, -17.19534466],
       [ 11.57562648,   0.8389633 , -13.4519955 ],
       [ 13.54675945,   0.29763646, -13.49240711],
       [ 11.41204109,   2.78266059, -12.83027801],
       [ 10.56158809,  -0.41323348, -12.19048092]])

In [10]:
qcmol = qcel.models.Molecule.from_file(DMSO)

In [11]:
qcmol.geometry

array([[  7.1407494 ,   1.34160357, -15.94150286],
       [  7.02969573,   3.27934588, -15.29029187],
       [  6.37365911,   0.06141398, -14.54136782],
       [  6.08497659,   1.14334413, -17.68106387],
       [ 10.36395491,   0.51005289, -16.60560116],
       [ 10.38402733,  -2.23374996, -17.19534466],
       [ 11.57562648,   0.8389633 , -13.4519955 ],
       [ 13.54675945,   0.29763646, -13.49240711],
       [ 11.41204109,   2.78266059, -12.83027801],
       [ 10.56158809,  -0.41323348, -12.19048092]])

In [13]:
from psiresp.grid import GridOptions

In [41]:
grid = GridOptions()._generate_vdw_grid(qcmol.symbols, qcmol.geometry)# / qcel.constants.conversion_factor("angstrom", "bohr"))

In [42]:
grid.shape

(1165, 3)

In [26]:
qcel.constants.conversion_factor("angstrom", "bohr")

1.8897261254578281

In [27]:
help(qcel.constants.conversion_factor)

Help on method conversion_factor in module qcelemental.physical_constants.context:

conversion_factor(base_unit: Union[str, ForwardRef('quantity._Quantity')], conv_unit: Union[str, ForwardRef('quantity._Quantity')]) -> float method of qcelemental.physical_constants.context.PhysicalConstantsContext instance
    Provides the conversion factor from one unit to another.
    
    The conversion factor is based on the current contexts CODATA.
    
    Parameters
    ----------
    base_unit
        The original units
    conv_unit
        The units to convert to
    
    Examples
    --------
    
    >>> conversion_factor("meter", "picometer")
    1e-12
    
    >>> conversion_factor("feet", "meter")
    0.30479999999999996
    
    >>> conversion_factor(10 * ureg.feet, "meter")
    3.0479999999999996
    
    Returns
    -------
    float
        The requested conversion factor



In [31]:
def surface(n):
    """Computes approximately n points on unit sphere. Code adapted from GAMESS.
    Parameters
    ----------
    n : int
        approximate number of requested surface points
    Returns
    -------
    ndarray
        numpy array of xyz coordinates of surface points
    """

    u = []
    eps = 1e-10
    nequat = int(np.sqrt(np.pi*n))
    nvert = int(nequat/2)
    nu = 0
    for i in range(nvert+1):
        fi = np.pi*i/nvert
        z = np.cos(fi)
        xy = np.sin(fi)
        nhor = int(nequat*xy+eps)
        if nhor < 1:
            nhor = 1
        for j in range(nhor):
            fj = 2*np.pi*j/nhor
            x = np.cos(fj)*xy
            y = np.sin(fj)*xy
            if nu >= n:
                return np.array(u)
            nu += 1
            u.append([x, y, z])
    return np.array(u)

In [28]:
def vdw_surface(coordinates, elements, scale_factor, density, input_radii):
    """Computes points outside the van der Waals surface of molecules.
    Parameters
    ----------
    coordinates : ndarray
        cartesian coordinates of the nuclei, in units of angstrom
    elements : list
        The symbols (e.g. C, H) for the atoms
    scale_factor : float
        The points on the molecular surface are set at a distance of
        scale_factor * vdw_radius away from each of the atoms.
    density : float
        The (approximate) number of points to generate per square angstrom
        of surface area. 1.0 is the default recommended by Kollman & Singh.
    input_radii : dict
        dictionary of user's defined VDW radii
    Returns
    -------
    radii : dict
        A dictionary of scaled VDW radii
    surface_points : ndarray
        array of the coordinates of the points on the surface
    """
    radii = {}
    surface_points = []
    # scale radii
    for i in elements:
        if i in radii.keys():
            continue
        if i in input_radii.keys():
            radii[i] = input_radii[i] * scale_factor
        elif i in vdw_r.keys():
            radii[i] = vdw_r[i] * scale_factor
        else:
            raise KeyError('%s is not a supported element; ' %i
                         + 'use the "VDW_RADII" option to add '
                         + 'its van der Waals radius.')
    # loop over atomic coordinates
    for i in range(len(coordinates)):
        # calculate approximate number of ESP grid points
        n_points = int(density * 4.0 * np.pi* np.power(radii[elements[i]], 2))
        # generate an array of n_points in a unit sphere around the atom
        dots = surface(n_points)
        # scale the unit sphere by the VDW radius and translate
        dots = coordinates[i] + radii[elements[i]] * dots
        for j in range(len(dots)):
            save = True
            for k in range(len(coordinates)):
                if i == k:
                    continue
                # exclude points within the scaled VDW radius of other atoms
                d = np.linalg.norm(dots[j] - coordinates[k])
                if d < radii[elements[k]]:
                    save = False
                    break
            if save:
                surface_points.append(dots[j])

    return np.array(surface_points), radii

In [52]:
def write(coordfile, gridfile):
    qcmol = qcel.models.Molecule.from_file(coordfile)
    options = GridOptions()
    points = []
    radii = options.get_vdwradii_for_elements(qcmol.symbols)
    for f in options.vdw_scale_factors:
        points.append(
            vdw_surface(qcmol.geometry, qcmol.symbols, f, 1.0, options.all_vdw_radii)[0]
        )
    point_arr = np.concatenate(points)
    gridpath = f"/Users/lily/pydev/psiresp/psiresp/tests/data/surfaces/{gridfile}"
    np.save(gridpath, point_arr)


In [54]:
write(DMSO_O2, "dmso_opt_c1_o2_grid.npy")